In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import glob

#讀訓練檔案
path = 'C:/Users/User/Downloads/train_set'
filelist = os.listdir(path)
train_df = pd.DataFrame(filelist)
#讀測試檔案
test_path = 'C:/Users/User/Downloads/test_set'
test_filelist = os.listdir(test_path)
test_df = pd.DataFrame(test_filelist)

#重新命名
train_df = train_df.rename(columns={0:'wav'})
test_df =  test_df.rename(columns={0:'wav'})

#讀 train csv
train_label=pd.read_csv('C:/Users/User/Downloads/train.csv')
train_df['label']=train_label['label']

#提取train特徵
def extract_features(files):
    file_name=os.path.join(os.path.abspath(path)+'/'+str(files.wav) )
    # 將音頻文件加載為浮點時間序列並分配默認採樣率
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    # 從時間序列生成梅爾頻率倒譜係數
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    # 生成短時傅立葉變換 (STFT) 以用於 chroma_stft
    stft = np.abs(librosa.stft(X))
    #從波形或功率譜圖計算色譜圖
    color = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    # 計算梅爾縮放的頻譜圖。
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    # 計算光譜對比度
    light = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    # 計算音調質心特徵
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
                                              sr=sample_rate).T, axis=0)
    return mfccs,color,mel,light,tonnetz
#提取test特徵
def test_extract_features(files):
    file_name=os.path.join(os.path.abspath(test_path)+'/'+str(files.wav) )
    # 將音頻文件加載為浮點時間序列並分配默認採樣率
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    # 從時間序列生成梅爾頻率倒譜係數
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    # 生成短時傅立葉變換 (STFT) 以用於 chroma_stft
    stft = np.abs(librosa.stft(X))
    #從波形或功率譜圖計算色譜圖
    color = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    # 計算梅爾縮放的頻譜圖。
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    # 計算光譜對比度
    light = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    # 計算音調質心特徵
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
                                              sr=sample_rate).T, axis=0)
    return mfccs,color,mel,light,tonnetz

#進行提取
train_features = train_df.apply(extract_features,axis=1)
test_features = test_df.apply(test_extract_features,axis=1)

#train特徵連接起來
features_train = []
for i in range(0, len(train_features)):
    features_train.append(np.concatenate((
        train_features[i][0],
        train_features[i][1],
        train_features[i][2],
        train_features[i][3],
        train_features[i][4]),axis=0))

#test特徵連起來
features_test = []
for i in range(0, len(test_features)):
    features_test.append(np.concatenate((
        test_features[i][0],
        test_features[i][1],
        test_features[i][2],
        test_features[i][3],
        test_features[i][4]),axis=0))

#x_train設為特徵numpy數組
X_train = np.array(features_train)
X_test =  np.array(features_test)

#y_train將label設為numpy
y_train = np.array(train_df['label'])

#one hot encoding
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
lb = LabelEncoder()
y_train = to_categorical(lb.fit_transform(y_train))

#建構模型
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping
model = Sequential()
model.add(Dense(193, input_dim=193, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100,verbose=1, mode='auto')
print(model.summary())

#執行
model.fit(X_train, y_train)

#測試集預測結果
predictions = model.predict(X_test)
predictions = model.predict_classes(X_test)
test_df['label'] = predictions
test_df.to_csv('C:/Users/User/Downloads/submission.csv',index=False)